# Notebook #3


### Name: Tulsi Patel


### Date: 11.01.2021


### PID: 730392259


### Goal: Filter and create a spike raster for Sst-IRES/Male/VISp/Natural Scenes in Brain Observatory 1.1. 

## Protocol

### Starting code. 

In [1]:
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

In [2]:
data_directory = '/Users/tulsipatel/local1/ecephys_cache_dir' 
# must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

In [3]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
cache.get_session_table().head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [5]:
sessions = cache.get_session_table()


print('Total number of sessions: ' + str(len(sessions)))

sessions.head()

Total number of sessions: 58


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [6]:
filtered_sessions = sessions[(sessions.sex == 'M') & \
                             (sessions.full_genotype.str.find('Sst-IRES') > -1) & \
                             (sessions.session_type == 'brain_observatory_1.1') & \
                             (['VISp' in acronyms for acronyms in 
                               sessions.ecephys_structure_acronyms])]

filtered_sessions.head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
756029989,2019-10-03T00:00:00Z,734865738,brain_observatory_1.1,96.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,684,2214,6,"[TH, DG, CA3, CA1, VISl, nan, PO, Eth, LP, VIS..."
758798717,2019-10-03T00:00:00Z,735109609,brain_observatory_1.1,102.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,593,1486,4,"[MGv, MGd, TH, DG, CA1, VISal, nan, SGN, LP, S..."
762602078,2019-10-03T00:00:00Z,744915204,brain_observatory_1.1,110.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,531,2233,6,"[MB, APN, NOT, DG, CA1, VISam, nan, VISrl, LGv..."


In [7]:
session = cache.get_session_data(filtered_sessions.index.values[0],
                                 isi_violations_maximum = np.inf,
                                 amplitude_cutoff_maximum = np.inf,
                                 presence_ratio_minimum = -np.inf
                                )

print([attr_or_method for attr_or_method in dir(session) if attr_or_method[0] != '_'])

Downloading:   0%|          | 0.00/2.86G [00:00<?, ?B/s]

['DETAILED_STIMULUS_PARAMETERS', 'LazyProperty', 'age_in_days', 'api', 'channel_structure_intervals', 'channels', 'conditionwise_spike_statistics', 'ecephys_session_id', 'from_nwb_path', 'full_genotype', 'get_current_source_density', 'get_inter_presentation_intervals_for_stimulus', 'get_invalid_times', 'get_lfp', 'get_parameter_values_for_stimulus', 'get_pupil_data', 'get_screen_gaze_data', 'get_stimulus_epochs', 'get_stimulus_parameter_values', 'get_stimulus_table', 'inter_presentation_intervals', 'invalid_times', 'mean_waveforms', 'metadata', 'num_channels', 'num_probes', 'num_stimulus_presentations', 'num_units', 'optogenetic_stimulation_epochs', 'presentationwise_spike_counts', 'presentationwise_spike_times', 'probes', 'rig_equipment_name', 'rig_geometry_data', 'running_speed', 'session_start_time', 'session_type', 'sex', 'specimen_name', 'spike_amplitudes', 'spike_times', 'stimulus_conditions', 'stimulus_names', 'stimulus_presentations', 'structure_acronyms', 'structurewise_unit_c

## Find the Natural Scenes segments in the session. 

In [8]:
session.get_stimulus_table(['natural_scenes']).head()

,frame,start_time,stimulus_block,stimulus_name,stop_time,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,
51353,5,5892.415053,9,natural_scenes,5892.665249,0.250196,4908
51354,114,5892.665249,9,natural_scenes,5892.915445,0.250196,4909
51355,13,5892.915445,9,natural_scenes,5893.165641,0.250196,4910
51356,70,5893.165641,9,natural_scenes,5893.415837,0.250196,4911
51357,26,5893.415837,9,natural_scenes,5893.666049,0.250211,4912


In [9]:
session.units.head()

,snr,presence_ratio,waveform_spread,cumulative_drift,isolation_distance,waveform_amplitude,firing_rate,nn_hit_rate,isi_violations,waveform_velocity_below,L_ratio,nn_miss_rate,waveform_halfwidth,waveform_repolarization_slope,amplitude_cutoff,waveform_PT_ratio,silhouette_score,waveform_recovery_slope,waveform_duration,local_index_unit,max_drift,waveform_velocity_above,cluster_id,peak_channel_id,d_prime,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
950910332,2.715757,0.94,40.0,389.93,5.258583e+01,120.868605,1.736711,0.908046,0.101473,NaN,0.001699,0.001221,0.604355,0.077617,0.139006,1.074955,0.060656,-0.004153,1.579564,3,30.07,0.000000,3,850264152,4.522811,NaN,NaN,6.034109,NaN,1.946667,0.933333,1.541009,1.093364,0.147841,0.054229,0.021446,NaN,0.016396,NaN,0.02,15.0,-0.133333,NaN,-0.898701,1.000000,-0.633333,270.0,60.0,0.829471,NaN,0.000843,0.355918,0.020954,NaN,4967,0.00,1.295036,0.0,1.603348,0.005482,2.909568,NaN,0.0145,False,False,False,False,False,0.060407,False,0.168823,NaN,0.068363,0.983471,0.049149,40,27,810755797,3,215.0,APN,8162.0,3538.0,6694.0,probeA,See electrode locations,29999.954846,1249.998119,True
950910326,2.619865,0.16,60.0,0.00,2.488905e+13,148.358438,0.003733,NaN,0.000000,NaN,NaN,0.000000,0.151089,0.549791,0.008088,1.861119,NaN,-0.288414,0.178559,2,NaN,-1.236181,2,850264152,4.345510,NaN,NaN,NaN,NaN,0.980000,NaN,0.958333,NaN,NaN,NaN,0.489796,NaN,0.000000,NaN,0.04,1.0,NaN,NaN,1.000000,NaN,0.400000,0.0,30.0,NaN,NaN,0.331333,NaN,0.734056,NaN,4927,0.00,0.000531,0.0,0.004030,0.000000,0.011990,NaN,0.1325,False,True,True,False,False,0.898305,True,NaN,NaN,0.957265,NaN,0.911164,40,27,810755797,3,215.0,APN,8162.0,3538.0,6694.0,probeA,See electrode locations,29999.954846,1249.998119,True
950910319,3.101263,0.92,20.0,411.00,8.244526e+01,141.769485,12.713519,0.987333,0.008115,NaN,0.000728,0.003420,0.164824,0.487856,0.028678,1.800296,0.080316,-0.315016,0.164824,1,62.53,0.000000,1,850264152,4.684774,NaN,200.0,23.995062,NaN,0.404958,2.044444,1.227261,0.459173,0.317774,0.062495,0.047684,10.000,1.164572,0.960,0.02,8.0,-0.607843,NaN,-0.048913,-0.556452,-0.084727,225.0,30.0,0.467734,NaN,0.465650,0.169705,0.547366,45.000,4966,0.75,1.228107,0.0,25.953140,4.778534,21.453399,NaN,0.0545,False,False,False,False,False,0.057492,False,0.309389,NaN,0.001913,0.110832,0.007163,40,27,810755797,3,215.0,APN,8162.0,3538.0,6694.0,probeA,See electrode locations,29999.954846,1249.998119,True
950910311,2.278383,0.99,50.0,265.32,5.509009e+01,112.067475,12.550090,0.970667,0.250391,NaN,0.012544,0.007522,0.151089,0.507110,0.500000,0.962239,NaN,-0.101162,0.260972,0,16.24,-0.343384,0,850264152,4.008943,NaN,700.0,1.194712,NaN,1.194902,2.737155,0.789116,0.300468,0.042230,0.033773,0.009639,82.857,0.769267,0.892,0.32,4.0,0.212577,NaN,-0.135346,0.072165,0.317887,180.0,60.0,0.072857,NaN,0.420964,0.778056,0.008967,37.143,4987,0.25,9.398309,0.0,9.977430,29.754545,9.593980,NaN,0.0115,False,False,False,False,False,0.150949,False,0.012523,NaN,0.006208,0

## Creating a Raster plot for spikes that occured in units that had a firing rate > 0 during the natural scenes set . 

In [10]:
# how many units have firing rates that are greater than 0?
print(f'{session.units.shape[0]} units total')
units_with_firing_rate = session.units[session.units['firing_rate'] > 0]
print(f'{units_with_firing_rate.shape[0]} units have firing rate > 0')

2073 units total
2073 units have firing rate > 0


In [11]:
 # grab an arbitrary unit (we made units_with_firing_rates above)
firing_rate_unit_ids = units_with_firing_rate.index.values
unit_id = firing_rate_unit_ids[0]

print(f"{len(session.spike_times[unit_id])} spikes were detected for unit {unit_id} at times:")
session.spike_times[unit_id]

/Users/tulsipatel/.local/lib/python3.8/site-packages/allensdk/brain_observatory/ecephys/ecephys_session.py:1093: UserWarning: Session includes invalid time intervals that could be accessed with the attribute 'invalid_times',Spikes within these intervals are invalid and may need to be excluded from the analysis.
  warnings.warn("Session includes invalid time intervals that could be accessed with the attribute 'invalid_times',"


14888 spikes were detected for unit 950910332 at times:


array([  51.63366736,  153.21330076,  153.85076847, ..., 9615.63732715,
       9615.75126066, 9615.80509407])

In [12]:
# get spike times from the first block of natural scenes presentations 
natural_scenes_presentation_ids = session.stimulus_presentations.loc[
    (session.stimulus_presentations['stimulus_name'] == 'natural_scenes')
].index.values

times = session.presentationwise_spike_times(
    stimulus_presentation_ids=natural_scenes_presentation_ids,
    unit_ids=firing_rate_unit_ids
)

times.head()

,stimulus_presentation_id,unit_id,time_since_stimulus_presentation_onset
spike_time,,,
5892.415103,51353,950951940,0.000050
5892.415156,51353,950916377,0.000103
5892.415168,51353,950930358,0.000115
5892.415201,51353,950924654,0.000148
5892.415503,51353,950945467,0.000450


## Raster Plot for Sst-IRES Male VISp Natural Scenes

In [ ]:
# get spike times from the first block of natural scenes presentations 
natural_scenes_presentation_ids = session.stimulus_presentations.loc[
    (session.stimulus_presentations['stimulus_name'] == 'natural_scenes')
].index.values

times = session.presentationwise_spike_times(
    stimulus_presentation_ids=natural_scenes_presentation_ids,
    unit_ids=firing_rate_unit_ids
)

times.head()

In [ ]:
first_natural_scene_presentation_id = times['stimulus_presentation_id'].values[0]
plot_times = times[times['stimulus_presentation_id'] == first_natural_scene_presentation_id]

fig = raster_plot(plot_times, title=f'spike raster for stimulus presentation {first_natural_scene_presentation_id}')
plt.show()

# also print out this presentation
session.stimulus_presentations.loc[first_natural_scene_presentation_id]